# Monetary and Fiscal Policy Coordination Notebook


In [66]:
import pandas as pd
import statsmodels as smf
import datetime as dt

from lets_plot import *
LetsPlot.setup_html()
from IPython.display import Image
import cairosvg

## Taylor Rule Deviation:

$i_t = 2 + \pi_t + a(\pi_t-\pi^*)+b(y_t - y_t^*)$

+ From the formula above we need:

    + Interest Rate
    + Inflation Rate
    + Targetted Inflation Rate
    + Potential Output

In [ ]:
# Getting my dataset for interest rates
df = pd.DataFrame()
for i in range(0,8): # For between 2013-2020.
    t = pd.read_excel(f'../data/interest-rates/Loan_Prime_Rate_Historical_Data ({i}).xlsx',engine='openpyxl')
    t = t.loc[~((t['Date']=='Data source:')|(t['Date'].isnull())),:]
    df = pd.concat([df,t], ignore_index=True)

# Getting the dataset for benchmark rates 2010-2013.
t = pd.read_excel('../data/interest-rates/cbpol.xlsx', engine = 'openpyxl', sheet_name = 'Monthly Series')
t.columns = t.loc[2]
t = t.drop([0,1,2])
t = t.loc[(~(t['M:CN'].isnull())&(t['Period']<dt.datetime(2013,10,25))&(t['Period']>dt.datetime(2009,12,31))),['Period','M:CN']]
t['Date'] = t['Period']
t['1Y'] = t['M:CN']
df = pd.concat([df,t], ignore_index=True)

# Cleaning the data up.
df['Date'] = pd.to_datetime(df['Date'])
df['1Y'] = df['1Y'].astype(float)
df = df.sort_values(by='Date',axis=0)
df = df.reset_index()
df = df.drop(columns =['Period', 'M:CN','5Y', 'index'])

df.to_csv('../data/interest-rates/chinese_LPR_rates_20102020.csv')
print(df)


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

           Date    1Y
0    2010-01-31  5.31
1    2010-02-28  5.31
2    2010-03-31  5.31
3    2010-04-30  5.31
4    2010-05-31  5.31
...         ...   ...
1510 2020-08-20  3.85
1511 2020-09-21  3.85
1512 2020-10-20  3.85
1513 2020-11-20  3.85
1514 2020-12-21  3.85

[1515 rows x 2 columns]


In [77]:
(
    ggplot(data = df, mapping = aes(x = 'Date', y = '1Y'))
    + geom_line()
)